# Merge Playoff Data with MoneyPuck Data

The MoneyPuck dataset contains a lot of information for each team and we would like to add our playoff data set to the table. First, we will need to remove the many unnecessary columns in the MoneyPuck dataset to make it easier to interpret. Then, we will merge the team names from the team_info.csv file to the MoneyPuck data since the playoff data refers to team name instead of team code. Finally, we will merge the MoneyPuck data with the Playoff Data based on season and team name.

### Clean MoneyPuck Data

In [1]:
# Import statements
import pandas as pd

In [2]:
# Loan MoneyPuck data
moneypuck = pd.read_excel('cleaned_moneypuck.xlsx')
moneypuck.head()

,team,season,position,situation,games_played,xGoalsPercentage,corsiPercentage,fenwickPercentage,iceTime,xOnGoalFor,...,scoreAdjustedShotsAttemptsAgainst,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst
0,MIN,2008,Team Level,other,82,0.48,0.49,0.49,11901,143.97,...,285.00,218,218.00,7,3.09,2.24,2.22,21.35,21.35,20.46
1,MIN,2008,Team Level,all,82,0.48,0.48,0.48,299195,2210.01,...,4490.31,3467,3459.72,257,36.30,27.99,29.58,204.64,204.38,200.25
2,MIN,2008,Team Level,5on5,82,0.46,0.47,0.47,230258,1602.93,...,3367.31,2575,2567.72,221,23.18,17.52,18.56,134.99,134.73,133.06
3,MIN,2008,Team Level,4on5,82,0.13,0.12,0.14,28293,65.18,...,735.00,579,579.00,1,9.34,7.21,7.36,43.14,43.14,41.59
4,MIN,2008,Team Level,5on4,82,0.88,0.88,0.86,28743,397.93,...,103.00,95,95.00,28,0.69,1.02,1.43,5.15,5.15,5.14


In [19]:
# Check shape of MoneyPuck data
moneypuck.shape

(1970, 19)

In [3]:
# Create list for moneypuck columns
full_columns = moneypuck.columns
full_columns

Index(['team', 'season', 'position', 'situation', 'games_played',
       'xGoalsPercentage', 'corsiPercentage', 'fenwickPercentage', 'iceTime',
       'xOnGoalFor',
       ...
       'scoreAdjustedShotsAttemptsAgainst', 'unblockedShotAttemptsAgainst',
       'scoreAdjustedUnblockedShotAttemptsAgainst', 'dZoneGiveawaysAgainst',
       'xGoalsFromxReboundsOfShotsAgainst',
       'xGoalsFromActualReboundsOfShotsAgainst', 'reboundxGoalsAgainst',
       'totalShotCreditAgainst', 'scoreAdjustedTotalShotCreditAgainst',
       'scoreFlurryAdjustedTotalShotCreditAgainst'],
      dtype='object', length=105)

In [4]:
# Create list of columns to keep
keep_columns = ['team', 'season', 'situation', 'games_played', 'iceTime', 'xGoalsFor', 
                'shotsOnGoalFor', 'goalsFor', 'penaltiesFor', 'penaltyMinutesFor', 'hitsFor', 
               'takeawaysFor', 'xGoalsAgainst', 'shotsOnGoalAgainst', 'goalsAgainst', 
                'penaltiesAgainst','penaltyMinutesAgainst', 'hitsAgainst', 'takeawaysAgainst', ]

In [5]:
# Create a list of columns to drop
drop_columns = list(set(full_columns) - set(keep_columns))

In [6]:
# Remove unnecessary columns
moneypuck.drop(columns = drop_columns, inplace = True)
moneypuck.head()

,team,season,situation,games_played,iceTime,xGoalsFor,shotsOnGoalFor,goalsFor,penaltiesFor,penaltyMinutesFor,hitsFor,takeawaysFor,xGoalsAgainst,shotsOnGoalAgainst,goalsAgainst,penaltiesAgainst,penaltyMinutesAgainst,hitsAgainst,takeawaysAgainst
0,MIN,2008,other,82,11901,20.06,154,26,8,16,43,19,21.66,158,28,15,34,39,17
1,MIN,2008,all,82,299195,188.02,2258,214,353,847,1468,440,202.35,2518,197,394,919,1443,443
2,MIN,2008,5on5,82,230258,115.06,1630,122,303,736,1319,351,132.79,1867,136,326,776,1301,345
3,MIN,2008,4on5,82,28293,6.20,67,8,21,53,64,56,41.63,417,27,28,59,34,12
4,MIN,2008,5on4,82,28743,46.71,407,58,21,42,42,14,6.28,76,6,25,50,69,69


### Clean Team Codes Data

In [39]:
codes = pd.read_csv('team_info.csv', index_col = 0)
codes.head()

,franchiseId,shortName,teamName,abbreviation,link
team_id,,,,,
1,23,New Jersey,Devils,NJD,/api/v1/teams/1
4,16,Philadelphia,Flyers,PHI,/api/v1/teams/4
26,14,Los Angeles,Kings,LAK,/api/v1/teams/26
14,31,Tampa Bay,Lightning,TBL,/api/v1/teams/14
6,6,Boston,Bruins,BOS,/api/v1/teams/6


In [40]:
# Create full name column 

codes['FullName'] = codes['shortName'] + ' ' + codes['teamName']
codes

,franchiseId,shortName,teamName,abbreviation,link,FullName
team_id,,,,,,
1,23,New Jersey,Devils,NJD,/api/v1/teams/1,New Jersey Devils
4,16,Philadelphia,Flyers,PHI,/api/v1/teams/4,Philadelphia Flyers
26,14,Los Angeles,Kings,LAK,/api/v1/teams/26,Los Angeles Kings
14,31,Tampa Bay,Lightning,TBL,/api/v1/teams/14,Tampa Bay Lightning
6,6,Boston,Bruins,BOS,/api/v1/teams/6,Boston Bruins
3,10,NY Rangers,Rangers,NYR,/api/v1/teams/3,NY Rangers Rangers
5,17,Pittsburgh,Penguins,PIT,/api/v1/teams/5,Pittsburgh Penguins
17,12,Detroit,Red Wings,DET,/api/v1/teams/17,Detroit Red Wings
28,29,San Jose,Sharks,SJS,/api/v1/teams/28,San Jose Sharks


In [42]:
# Manually adjust names where necessary

codes['FullName'][3] = 'New York Rangers'
codes['FullName'][19] = 'St. Louis Blues'
codes['FullName'][2] = 'New York Islanders'
codes

<ipython-input-42-10635dd562f8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  codes['FullName'][3] = 'New York Rangers'
<ipython-input-42-10635dd562f8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  codes['FullName'][19] = 'St. Louis Blues'
<ipython-input-42-10635dd562f8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  codes['FullName'][2] = 'New York Islanders'


,franchiseId,shortName,teamName,abbreviation,link,FullName
team_id,,,,,,
1,23,New Jersey,Devils,NJD,/api/v1/teams/1,New Jersey Devils
4,16,Philadelphia,Flyers,PHI,/api/v1/teams/4,Philadelphia Flyers
26,14,Los Angeles,Kings,LAK,/api/v1/teams/26,Los Angeles Kings
14,31,Tampa Bay,Lightning,TBL,/api/v1/teams/14,Tampa Bay Lightning
6,6,Boston,Bruins,BOS,/api/v1/teams/6,Boston Bruins
3,10,NY Rangers,Rangers,NYR,/api/v1/teams/3,New York Rangers
5,17,Pittsburgh,Penguins,PIT,/api/v1/teams/5,Pittsburgh Penguins
17,12,Detroit,Red Wings,DET,/api/v1/teams/17,Detroit Red Wings
28,29,San Jose,Sharks,SJS,/api/v1/teams/28,San Jose Sharks


In [43]:
# Remove unnecessary columns
codes.drop(columns = ['franchiseId', 'shortName', 'teamName', 'link'], inplace = True)
codes.head()

,abbreviation,FullName
team_id,,
1,NJD,New Jersey Devils
4,PHI,Philadelphia Flyers
26,LAK,Los Angeles Kings
14,TBL,Tampa Bay Lightning
6,BOS,Boston Bruins


In [45]:
# Rename team code column
codes = codes.rename(columns = {'abbreviation': 'team'})
codes.head()

,team,FullName
team_id,,
1,NJD,New Jersey Devils
4,PHI,Philadelphia Flyers
26,LAK,Los Angeles Kings
14,TBL,Tampa Bay Lightning
6,BOS,Boston Bruins


### Merge MoneyPuck Data with Codes Data

In [46]:
moneypuck_merged = moneypuck.merge(codes, on = 'team', how = 'left')
moneypuck_merged

,team,season,situation,games_played,iceTime,xGoalsFor,shotsOnGoalFor,goalsFor,penaltiesFor,penaltyMinutesFor,hitsFor,takeawaysFor,xGoalsAgainst,shotsOnGoalAgainst,goalsAgainst,penaltiesAgainst,penaltyMinutesAgainst,hitsAgainst,takeawaysAgainst,FullName
0,MIN,2008,other,82,11901,20.06,154,26,8,16,43,19,21.66,158,28,15,34,39,17,Minnesota Wild
1,MIN,2008,all,82,299195,188.02,2258,214,353,847,1468,440,202.35,2518,197,394,919,1443,443,Minnesota Wild
2,MIN,2008,5on5,82,230258,115.06,1630,122,303,736,1319,351,132.79,1867,136,326,776,1301,345,Minnesota Wild
3,MIN,2008,4on5,82,28293,6.20,67,8,21,53,64,56,41.63,417,27,28,59,34,12,Minnesota Wild
4,MIN,2008,5on4,82,28743,46.71,407,58,21,42,42,14,6.28,76,6,25,50,69,69,Minnesota Wild
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1965,NYI,2020,other,56,6649,14.09,70,9,4,8,24,10,17.74,79,16,6,12,22,17,New York Islanders
1966,NYI,2020,all,56,205063,148.64,1624,152,145,314,1455,269,129.63,1590,125,153,326,1264,321,New York Islanders
1967,NYI,2020,5on5,56,171310,106.18,1325,113,125,272,1371,231,91.72,1307,90,138,296,1180,268,New York Islanders
1968,NYI,2020,4on5,56,12840,3.76,34,4,9,18,38,23,17.38,169,19,9,18,12,7,New York Islanders


In [49]:
# Check if there are any null team names
moneypuck_merged['FullName'].value_counts()

Detroit Red Wings        65
Dallas Stars             65
Anaheim Ducks            65
Columbus Blue Jackets    65
Nashville Predators      65
New York Islanders       65
St. Louis Blues          65
Montreal Canadiens       65
Calgary Flames           65
Philadelphia Flyers      65
Vancouver Canucks        65
Washington Capitals      65
Toronto Maple Leafs      65
Edmonton Oilers          65
Minnesota Wild           65
Carolina Hurricanes      65
Ottawa Senators          65
New York Rangers         65
Boston Bruins            65
Florida Panthers         65
Colorado Avalanche       65
Chicago Blackhawks       65
Arizona Coyotes          65
Buffalo Sabres           65
Pittsburgh Penguins      65
Winnipeg Jets            50
Vegas Golden Knights     20
Atlanta Thrashers        15
Name: FullName, dtype: int64

### Clean Playoffs Data

In [50]:
# Load playoffs dataset
playoffs = pd.read_excel('Playoffs data - 12.4.21.xlsx', sheet_name = 'playoffs')
playoffs

,Year,Team,Playoffs,Quarterfinals,Semifinals,Finals,Winner
0,2008-2009,Boston Bruins,1,1.0,NaN,NaN,NaN
1,2008-2009,Washington Capitals,1,1.0,NaN,NaN,NaN
2,2008-2009,Carolina Hurricanes,1,1.0,1.0,NaN,NaN
3,2008-2009,Pittsburgh Penguins,1,1.0,1.0,1.0,1.0
4,2008-2009,Anaheim Ducks,1,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
203,2020-2021,Florida Panthers,1,NaN,NaN,NaN,NaN
204,2020-2021,St. Louis Blues,1,NaN,NaN,NaN,NaN
205,2020-2021,Minnesota Wild,1,NaN,NaN,NaN,NaN
206,2020-2021,Toronto Maple Leafs,1,NaN,NaN,NaN,NaN


In [51]:
# Replace NaN values with zero
playoffs = playoffs.fillna(0)
playoffs.head()

,Year,Team,Playoffs,Quarterfinals,Semifinals,Finals,Winner
0,2008-2009,Boston Bruins,1,1.0,0.0,0.0,0.0
1,2008-2009,Washington Capitals,1,1.0,0.0,0.0,0.0
2,2008-2009,Carolina Hurricanes,1,1.0,1.0,0.0,0.0
3,2008-2009,Pittsburgh Penguins,1,1.0,1.0,1.0,1.0
4,2008-2009,Anaheim Ducks,1,1.0,0.0,0.0,0.0


In [52]:
# Check data count is accurate
playoffs.groupby('Year').sum()

,Playoffs,Quarterfinals,Semifinals,Finals,Winner
Year,,,,,
2008-2009,16,8.0,4.0,2.0,1.0
2009-2010,16,8.0,4.0,2.0,1.0
2010-2011,16,8.0,4.0,2.0,1.0
2011-2012,16,8.0,4.0,2.0,1.0
2012-2013,16,8.0,4.0,2.0,1.0
2013-2014,16,8.0,4.0,2.0,1.0
2014-2015,16,8.0,4.0,2.0,1.0
2015-2016,16,8.0,4.0,2.0,1.0
2016-2017,16,8.0,4.0,2.0,1.0


In [58]:
# Create season column based on year
playoffs['season'] = [playoffs['Year'][i][0:4] for i in range(len(playoffs))]
playoffs['season'] = playoffs['season'].astype(int)
playoffs.head()

,Year,TeamName,Playoffs,Quarterfinals,Semifinals,Finals,Winner,season
0,2008-2009,Boston Bruins,1,1.0,0.0,0.0,0.0,2008
1,2008-2009,Washington Capitals,1,1.0,0.0,0.0,0.0,2008
2,2008-2009,Carolina Hurricanes,1,1.0,1.0,0.0,0.0,2008
3,2008-2009,Pittsburgh Penguins,1,1.0,1.0,1.0,1.0,2008
4,2008-2009,Anaheim Ducks,1,1.0,0.0,0.0,0.0,2008


In [60]:
# Rename columns
playoffs.rename(columns = {'TeamName': 'FullName'}, inplace = True)
playoffs.head()

,Year,FullName,Playoffs,Quarterfinals,Semifinals,Finals,Winner,season
0,2008-2009,Boston Bruins,1,1.0,0.0,0.0,0.0,2008
1,2008-2009,Washington Capitals,1,1.0,0.0,0.0,0.0,2008
2,2008-2009,Carolina Hurricanes,1,1.0,1.0,0.0,0.0,2008
3,2008-2009,Pittsburgh Penguins,1,1.0,1.0,1.0,1.0,2008
4,2008-2009,Anaheim Ducks,1,1.0,0.0,0.0,0.0,2008


### Merge Moneypuck Data with Playoffs Data

In [61]:
data = moneypuck_merged.merge(playoffs, on = ['FullName', 'season'], how = 'left')
data

,team,season,situation,games_played,iceTime,xGoalsFor,shotsOnGoalFor,goalsFor,penaltiesFor,penaltyMinutesFor,...,penaltyMinutesAgainst,hitsAgainst,takeawaysAgainst,FullName,Year,Playoffs,Quarterfinals,Semifinals,Finals,Winner
0,MIN,2008,other,82,11901,20.06,154,26,8,16,...,34,39,17,Minnesota Wild,NaN,NaN,NaN,NaN,NaN,NaN
1,MIN,2008,all,82,299195,188.02,2258,214,353,847,...,919,1443,443,Minnesota Wild,NaN,NaN,NaN,NaN,NaN,NaN
2,MIN,2008,5on5,82,230258,115.06,1630,122,303,736,...,776,1301,345,Minnesota Wild,NaN,NaN,NaN,NaN,NaN,NaN
3,MIN,2008,4on5,82,28293,6.20,67,8,21,53,...,59,34,12,Minnesota Wild,NaN,NaN,NaN,NaN,NaN,NaN
4,MIN,2008,5on4,82,28743,46.71,407,58,21,42,...,50,69,69,Minnesota Wild,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1965,NYI,2020,other,56,6649,14.09,70,9,4,8,...,12,22,17,New York Islanders,2020-2021,1.0,1.0,1.0,0.0,0.0
1966,NYI,2020,all,56,205063,148.64,1624,152,145,314,...,326,1264,321,New York Islanders,2020-2021,1.0,1.0,1.0,0.0,0.0
1967,NYI,2020,5on5,56,171310,106.18,1325,113,125,272,...,296,1180,268,New York Islanders,2020-2021,1.0,1.0,1.0,0.0,0.0
1968,NYI,2020,4on5,56,12840,3.76,34,4,9,18,...,18,12,7,New York Islanders,2020-2021,1.0,1.0,1.0,0.0,0.0


In [62]:
# Replace NaN values with zero for merged playoff columns
data = data.fillna(0)
data.head()

,team,season,situation,games_played,iceTime,xGoalsFor,shotsOnGoalFor,goalsFor,penaltiesFor,penaltyMinutesFor,...,penaltyMinutesAgainst,hitsAgainst,takeawaysAgainst,FullName,Year,Playoffs,Quarterfinals,Semifinals,Finals,Winner
0,MIN,2008,other,82,11901,20.06,154,26,8,16,...,34,39,17,Minnesota Wild,0,0.0,0.0,0.0,0.0,0.0
1,MIN,2008,all,82,299195,188.02,2258,214,353,847,...,919,1443,443,Minnesota Wild,0,0.0,0.0,0.0,0.0,0.0
2,MIN,2008,5on5,82,230258,115.06,1630,122,303,736,...,776,1301,345,Minnesota Wild,0,0.0,0.0,0.0,0.0,0.0
3,MIN,2008,4on5,82,28293,6.20,67,8,21,53,...,59,34,12,Minnesota Wild,0,0.0,0.0,0.0,0.0,0.0
4,MIN,2008,5on4,82,28743,46.71,407,58,21,42,...,50,69,69,Minnesota Wild,0,0.0,0.0,0.0,0.0,0.0


In [63]:
# Write merged data to a new .csv file
data.to_csv('Merged_Moneypuck_data.csv')